In [8]:
import pandas as pd
import numpy as np
import os

In [10]:

import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, inspect

# Data
connecting to the text message database using sql alchemy.

**** thoughts -  will I have to store this data in a separte table/db after cleaning? seems like an extra step if its alreayd all in one place.unsure if when making public I will be able to query my personal data. Also - longevity. 
do these texts die?

In [11]:
engine = db.create_engine('sqlite:////Users/michaelharnett/Library/Messages/chat.db')

In [12]:
engine

Engine(sqlite:////Users/michaelharnett/Library/Messages/chat.db)

In [13]:
connection = engine.connect()

In [14]:
metadata = db.MetaData()

In [15]:
metadata

MetaData()

In [16]:
db.tables()

AttributeError: module 'sqlalchemy' has no attribute 'tables'

In [17]:
metadata.tables.keys()

dict_keys([])

In [18]:
engine.table_names()

/var/folders/xl/ftndz_9j3hn5ybffb6t5q42m0000gn/T/ipykernel_3858/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['_SqliteDatabaseProperties',
 'attachment',
 'chat',
 'chat_handle_join',
 'chat_message_join',
 'deleted_messages',
 'handle',
 'kvtable',
 'message',
 'message_attachment_join',
 'message_processing_task',
 'sqlite_sequence',
 'sqlite_stat1',
 'sync_deleted_attachments',
 'sync_deleted_chats',
 'sync_deleted_messages']

In [19]:
insp = inspect(engine)
insp

In [20]:
db.Table()

AttributeError: 'Table' object has no attribute 'name'

In [21]:
# pulling in query I found online/worked on in DB Browser. For somereason
#I could not get it to work unless all on one line, so it is one long gross line.

orig_df = pd.read_sql("select distinct m.rowid ,m.is_from_me IsFromMe ,case when m.is_from_me = 1 then m.account else h.id end as FromPhoneNumber ,datetime((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') as TextDate, m.text MessageText,c.display_name RoomName from message as m left join handle as h on m.handle_id = h.rowid left join chat as c on m.cache_roomnames = c.room_name left join chat_handle_join as ch on c.rowid = ch.chat_id left join handle as h2 on ch.handle_id = h2.rowid where RoomName LIKE 'Wordle Fam%' order by m.date desc;", engine)

# EDA BABY


In [22]:
df = orig_df[['FromPhoneNumber','TextDate','MessageText']] # taking only the fields I need

In [23]:


df = df.rename(columns = {'FromPhoneNumber':'player', 'TextDate':'date', 'MessageText':'message'})

In [24]:
df

,player,date,message
0,E:michaelcharnett@gmail.com,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1,+12017253072,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2,+15514868670,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3,+12014523662,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
4,+15514868670,2022-04-08 13:31:09,Does anyone know the fios login
...,...,...,...
683,E:michaelcharnett@gmail.com,2022-02-03 19:12:58,I got wordle in 2 today 😏
684,E:michaelcharnett@gmail.com,2021-07-20 13:36:50,￼
685,+12017253072,2021-07-20 13:22:56,Thank you😊
686,+12014523662,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95


In [25]:
# Renaming changing the Id from phone number to our names

df.loc[df.player == '+12014523662', 'player'] = 'Danielle'
df.loc[df.player == '+12017253072', 'player'] = 'Dad'
df.loc[df.player == '+15514868670', 'player'] = 'Chris'
df.loc[df.player == 'E:michaelcharnett@gmail.com', 'player'] = 'Michael'

In [26]:
df.message.str.contains('Wordle')

0       True
1       True
2       True
3       True
4      False
       ...  
683    False
684    False
685    False
686    False
687    False
Name: message, Length: 688, dtype: object

In [27]:
df.player.value_counts()

Michael     198
Danielle    198
Dad         169
Chris       119
Name: player, dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   player   684 non-null    object
 1   date     688 non-null    object
 2   message  684 non-null    object
dtypes: object(3)
memory usage: 16.2+ KB


#### trying to figure the best way to filter the messages
only want this to look at the messages that we send from a completed game. they all start Wordle ### with the game number. Im glad that number is included, it will be easier to join on that!

In [29]:
df.message.astype('str')[3][:5]=='Wordl'

True

In [30]:
df.message[3][:5]=='Wordl'

True

In [31]:
df[df.player == 'None']

,player,date,message


In [32]:
df = df.dropna()

In [33]:
df[df.message.str.startswith("Wor")]

,player,date,message
0,Michael,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
5,Chris,2022-04-08 08:34:31,Wordle 293 4/6\n\n⬛️🟨🟩🟨⬛️\n🟩⬛️🟩⬛️🟩\n🟩⬛️🟩🟩🟩\n🟩🟩🟩🟩🟩
...,...,...,...
672,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩
673,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩
676,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...
677,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩


In [34]:
df[df.message.str.contains('Wordle')]

,player,date,message
0,Michael,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
5,Chris,2022-04-08 08:34:31,Wordle 293 4/6\n\n⬛️🟨🟩🟨⬛️\n🟩⬛️🟩⬛️🟩\n🟩⬛️🟩🟩🟩\n🟩🟩🟩🟩🟩
...,...,...,...
673,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩
675,Dad,2022-02-05 01:08:40,￼Wordle 231 3/6\n\n3 for me \n\n⬜🟨⬜🟨⬜\n🟨🟩🟩🟨🟩\n...
676,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...
677,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩


In [35]:
#df[df.message.str.contains('🟩')] # interesting i'm surprised this works!

## 

#df[df.message.str.contains('Wordle') & df.message.str.contains('🟩')] # really should use this, but unsure if the green chracter will trhow error in production

In [36]:
# creating a game number and score coluns

for rows in df:
    df['gamenumber'] = df.message[:6]

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [37]:
df = df.drop(columns=['gamenumber'])

In [38]:
df


,player,date,message
0,Michael,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
4,Chris,2022-04-08 13:31:09,Does anyone know the fios login
...,...,...,...
683,Michael,2022-02-03 19:12:58,I got wordle in 2 today 😏
684,Michael,2021-07-20 13:36:50,￼
685,Dad,2021-07-20 13:22:56,Thank you😊
686,Danielle,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95


In [39]:
df.iloc[2]['message'][11:15]

'3/6 '

In [40]:

## applying the funtion did not work since there was nothing keeping it at row level the whoe 
## df had the data from the last entry 

# def game_info_grabber (message):
#     '''
#     going to make a mini function to seperate the messages. I am going to make one colum
#     just the first 6 charachters. all messages I want should start with 'Wordle' this 
#     will allow another layer of filtering, making sure I am not capturing messages that
#     we were just using the word 'wordle' in
    
#     next will be game number,
    
#     then score.
#     '''
#     df['wordle_check'] = message[:6]
#     df['game_num'] = message[7:10]
#     df['score'] = message[11:14]    

In [41]:
df.message.apply(game_info_grabber)

NameError: name 'game_info_grabber' is not defined

In [42]:
df

,player,date,message
0,Michael,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
4,Chris,2022-04-08 13:31:09,Does anyone know the fios login
...,...,...,...
683,Michael,2022-02-03 19:12:58,I got wordle in 2 today 😏
684,Michael,2021-07-20 13:36:50,￼
685,Dad,2021-07-20 13:22:56,Thank you😊
686,Danielle,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95


In [43]:
df['wordle_check'] = message[:6]
df['game_num'] = message[7:10]
df['score'] = message[11:14]    

NameError: name 'message' is not defined

In [44]:
df = df.drop(columns = ['wordle_check','game_num','score'])

KeyError: "['wordle_check' 'game_num' 'score'] not found in axis"

In [45]:
df.message[:6]

0                Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
1                Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩
2             Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩
3         Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩
4                      Does anyone know the fios login
5    Wordle 293 4/6\n\n⬛️🟨🟩🟨⬛️\n🟩⬛️🟩⬛️🟩\n🟩⬛️🟩🟩🟩\n🟩🟩🟩🟩🟩
Name: message, dtype: object

In [46]:
for rows in df:
    print(rows)

player
date
message


In [47]:
df['wordle_check'] = ''
df['game_num'] = ''
df['score'] = ''

In [48]:
## the loop below was breaking at index 540...I had removed nulls and
## didn't reset the index.
df = df.dropna()
df = df.reset_index()
df =df.drop(columns = ['index'])

In [49]:
df

,player,date,message,wordle_check,game_num,score
0,Michael,2022-04-09 09:01:30,Wordle 294 3/6\n\n⬛🟨🟨⬛🟨\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩,,,
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩,,,
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩,,,
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩,,,
4,Chris,2022-04-08 13:31:09,Does anyone know the fios login,,,
...,...,...,...,...,...,...
679,Michael,2022-02-03 19:12:58,I got wordle in 2 today 😏,,,
680,Michael,2021-07-20 13:36:50,￼,,,
681,Dad,2021-07-20 13:22:56,Thank you😊,,,
682,Danielle,2021-07-20 13:22:22,Yes it’s my email and password is Lucky95,,,


In [50]:
for i in range(1,len(df)):
    df.loc[i,'wordle_check'] = df.loc[i,'message'][:6]
    df.loc[i,'game_num'] = df.loc[i,'message'][7:10]
    df.loc[i,'score'] = df.loc[i,'message'][11:14]
    

    

In [51]:
df[535:545]

,player,date,message,wordle_check,game_num,score
535,Dad,2022-02-23 00:57:34,Wordle 249 4/6\n\n⬜⬜⬜⬜🟩\n⬜🟩⬜⬜🟩\n⬜🟩🟩🟨🟩\n🟩🟩🟩🟩🟩,Wordle,249,4/6
536,Michael,2022-02-23 00:18:22,Wordle 249 6/6\n\n🟨🟩⬛⬛⬛\n🟨🟩⬛🟨⬛\n⬛🟩🟩⬛🟩\n⬛🟩🟩🟩🟩\n...,Wordle,249,6/6
537,Michael,2022-02-23 00:10:11,Laughed at “https://www.instagram.com/reel/CaN...,Laughe,at,“ht
538,Danielle,2022-02-23 00:05:29,Wordle 249 5/6\n\n⬛⬛🟩⬛🟩\n⬛⬛🟩🟩🟩\n⬛🟩🟩🟩🟩\n⬛🟩🟩🟩🟩\n...,Wordle,249,5/6
539,Danielle,2022-02-22 23:51:43,https://www.instagram.com/reel/CaNVcaADIBu/?ut...,https:,/ww,.in
540,Michael,2022-02-22 08:21:53,Wordle 248 3/6\n\n⬛⬛⬛🟨⬛\n🟩🟨⬛⬛🟩\n🟩🟩🟩🟩🟩\n\nThe w...,Wordle,248,3/6
541,Chris,2022-02-22 07:59:23,Wordle 248 5/6\n\n⬛️⬛️⬛️⬛️🟨\n🟨⬛️⬛️⬛️⬛️\n⬛️⬛️🟨🟨...,Wordle,248,5/6
542,Dad,2022-02-22 00:50:07,Wordle 248 3/6\n\n⬜🟨⬜⬜⬜\n🟨⬜⬜🟨🟨\n🟩🟩🟩🟩🟩,Wordle,248,3/6
543,Danielle,2022-02-22 00:19:42,Wordle 248 3/6\n\n⬛⬛⬛🟩🟨\n🟩🟩⬛🟩⬛\n🟩🟩🟩🟩🟩\n\nMy 3 ...,Wordle,248,3/6
544,Michael,2022-02-21 15:42:11,Wordle 247 3/6\n\n⬛🟨⬛🟨⬛\n🟨🟨🟨⬛🟨\n🟩🟩🟩🟩🟩\n\nWooo,Wordle,247,3/6


In [52]:
wordledf = df[df.wordle_check == 'Wordle']
wordledf

,player,date,message,wordle_check,game_num,score
1,Dad,2022-04-09 01:01:55,Wordle 294 3/6\n\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟩⬜\n🟩🟩🟩🟩🟩,Wordle,294,3/6
2,Chris,2022-04-09 00:37:30,Wordle 294 3/6 \n\n🟩🟩⬛️🟨🟨\n🟩🟩🟩🟨⬛️\n🟩🟩🟩🟩🟩,Wordle,294,3/6
3,Danielle,2022-04-09 00:35:01,Wordle 294 4/6\n\n⬛⬛⬛🟨⬛\n⬛🟨🟩🟨⬛\n🟩🟩🟩🟨⬛\n🟩🟩🟩🟩🟩,Wordle,294,4/6
5,Chris,2022-04-08 08:34:31,Wordle 293 4/6\n\n⬛️🟨🟩🟨⬛️\n🟩⬛️🟩⬛️🟩\n🟩⬛️🟩🟩🟩\n🟩🟩🟩🟩🟩,Wordle,293,4/6
6,Michael,2022-04-08 07:46:15,Wordle 293 5/6\n\n⬛🟨🟩🟩⬛\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n🟩⬛🟩🟩🟩\n...,Wordle,293,5/6
...,...,...,...,...,...,...
668,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4/6
669,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4/6
672,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5/6
673,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4/6


In [212]:
# INTERESTING 
'''
was curious to see if python would consider this the same character. it does not! Maybe I can implement total yellows too,
like correctly guessed letters
'''


'🟨' == '🟩'

False

In [181]:
wordledf

,player,date,message,wordle_check,game_num,score
1,Chris,2022-03-26 09:10:04,Wordle 280 4/6\n\n⬛️🟨⬛️⬛️⬛️\n⬛️⬛️🟨⬛️⬛️\n⬛️🟨⬛️🟨...,Wordle,280,4/6
3,Danielle,2022-03-26 00:50:01,Wordle 280 5/6\n\n⬛⬛🟨⬛⬛\n⬛⬛🟩⬛🟨\n🟩⬛🟩⬛⬛\n🟩⬛🟩⬛🟩\n...,Wordle,280,5/6
4,Dad,2022-03-26 00:42:45,Wordle 280 3/6\n\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩,Wordle,280,3/6
9,Michael,2022-03-25 19:05:29,Wordle 279 4/6\n\n🟨🟩⬛⬛⬛\n⬛🟩⬛🟨⬛\n🟨🟩🟩⬛🟨\n🟩🟩🟩🟩🟩\n...,Wordle,279,4/6
10,Dad,2022-03-25 01:05:31,Wordle 279 3/6\n\n⬜⬜⬜⬜⬜\n🟨🟨⬜🟨⬜\n🟩🟩🟩🟩🟩,Wordle,279,3/6
...,...,...,...,...,...,...
571,Danielle,2022-02-05 11:54:27,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n⬛🟨⬛⬛🟩\n🟩⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Wordle,231,4/6
572,Michael,2022-02-05 10:31:55,Wordle 231 4/6\n\n🟨⬛🟨⬛⬛\n🟩🟩⬛⬛⬛\n🟩🟩🟩⬛🟨\n🟩🟩🟩🟩🟩,Wordle,231,4/6
575,Michael,2022-02-04 08:41:31,Wordle 230 5/6\n\n🟨🟨⬛⬛⬛\n🟨🟩⬛⬛🟨\n⬛🟩🟩⬛🟨\n🟩🟩🟩🟩⬛\n...,Wordle,230,5/6
576,Danielle,2022-02-04 08:39:54,Wordle 230 4/6\n\n⬛🟨⬛⬛🟨\n⬛🟨🟨🟨⬛\n🟩🟨🟨🟨🟩\n🟩🟩🟩🟩🟩,Wordle,230,4/6


In [53]:
wordledf = wordledf.reset_index()
wordledf = wordledf.drop(columns = ['index'])

In [54]:
### changing the score, to just the actual tires used, since they're all out of 6 guesses.\

for i in range(len(wordledf)):
    wordledf.loc[i, 'score'] = wordledf.loc[i, 'score'][0]

In [55]:
#dropping mesage column, imprtant datahas been extracted.

#also date column? date is not as important, if we're matching on game number, can be undone!

######ALTHOUGH super long term, we can maybe  

wordledf = wordledf.drop(columns = ['message', 'date'])
wordledf = wordledf.drop(columns = ['wordle_check'])

In [56]:
wordledf # TADA! this should be everything needed to start modeling really


,player,game_num,score
0,Dad,294,3
1,Chris,294,3
2,Danielle,294,4
3,Chris,293,4
4,Michael,293,5
...,...,...,...
228,Danielle,231,4
229,Michael,231,4
230,Michael,230,5
231,Danielle,230,4


# visualizations and extrapolations


In [48]:
MichaelDF = wordledf[wordledf.player == 'Michael']
MichaelDF

,player,game_num,score


In [68]:
wordledf.player == 'Michael'


### thought this was a data type mismatch issue, BUTTTT it wound up being an extra space
### after I typed Michael the first time....fixed thtoughout the module.

0      False
1      False
2      False
3      False
4      False
       ...  
177    False
178    False
179    False
180    False
181    False
Name: player, Length: 182, dtype: bool

###### the fuck


In [63]:
wordledf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   player    182 non-null    object
 1   game_num  182 non-null    object
 2   score     182 non-null    object
dtypes: object(3)
memory usage: 4.4+ KB


In [57]:
wordledf.player = wordledf.player.astype('str')
wordledf.game_num = wordledf.game_num.astype('int')
wordledf.score = wordledf.score.astype('str')

In [59]:
wordledf.player.value_counts()

Danielle    64
Dad         61
Chris       55
Michael     53
Name: player, dtype: int64

In [68]:
michaeldf = wordledf[wordledf.player == 'Michael']

danielledf = wordledf[wordledf.player == 'Danielle']

chrisdf = wordledf[wordledf.player == 'Chris']

daddf = wordledf[wordledf.player == 'Dad']


In [61]:
wordledf['LOSS'] = ''

In [62]:
wordledf[wordledf.score == 'x']
wordledf[wordledf.score == 'X']    #there seems to be case inconsistencies 

wordledf.loc[wordledf.score == 'x','LOSS'] = 1
wordledf.loc[wordledf.score != 'x','LOSS'] = 0

wordledf.loc[wordledf.score == 'X','LOSS'] = 1
wordledf.loc[wordledf.score != 'X','LOSS'] = 0

In [63]:
wordledf.score.value_counts()

4    75
5    70
3    47
6    22
2    10
X     7
x     2
Name: score, dtype: int64

In [69]:
michaeldf = michaeldf.sort_values(by='game_num', ascending = False)
danielledf = danielledf.sort_values(by='game_num', ascending = False)
chrisdf = chrisdf.sort_values(by='game_num', ascending = False)
daddf = daddf.sort_values(by='game_num', ascending = False)

In [70]:
michaeldf.head(5)

,player,game_num,score,LOSS
4,Michael,293,5,0
10,Michael,291,6,0
15,Michael,290,4,0
27,Michael,286,6,0
32,Michael,285,4,0


In [71]:
print(wordledf.game_num.min())
print(wordledf.game_num.max())

230
294


# duh

just remembering I can make masks and don't have to keep typing this out or copy and pasting
a;gksadjg;lksg

In [67]:
scores_mask = (michaeldf.score != 'X' & michaeldf.score != 'x')

NameError: name 'michaeldf' is not defined

In [62]:
michaeldf[michaeldf.score != 'X']

,player,game_num,score,LOSS
3,Michael,279,4,0.0
8,Michael,278,5,0.0
16,Michael,276,3,0.0
28,Michael,271,6,0.0
32,Michael,270,4,0.0
36,Michael,269,6,0.0
42,Michael,268,5,0.0
44,Michael,267,5,0.0
55,Michael,264,4,0.0
59,Michael,263,3,0.0


### coming back into it,

I don't remember what I was attempting with the mask


# WAIT
### lets change gears

or. a better way to just look at numbers, so I can do an average 

In [66]:
michaeldf.score.isnumeric

AttributeError: 'Series' object has no attribute 'isnumeric'

In [85]:
pd.to_numeric(michaeldf['score'], errors='coerce').notnull().all()

False

In [113]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

In [95]:
len(michaelwindf)

40

In [96]:
len(michaeldf)

44

michael_winloss = (len(michaelwindf)/(len(michaeldf)-len(michaelwindf)))
michael_winloss

In [185]:
michael_winpercent = (len(michaelwindf)/len(michaeldf)*100)
michael_winpercent

90.9090909090909

# Making a df of just wins, for all aggregates 

In [198]:
michaelwindf = michaeldf[michaeldf['score'].astype(str).str.isnumeric()]

dadwindf = daddf[daddf['score'].astype(str).str.isnumeric()]

daniellewindf = danielledf[danielledf['score'].astype(str).str.isnumeric()]

chriswindf = chrisdf[chrisdf['score'].astype(str).str.isnumeric()]


In [187]:
michaelwindf

,player,game_num,score,LOSS
3,Michael,279,4,0.0
8,Michael,278,5,0.0
16,Michael,276,3,0.0
28,Michael,271,6,0.0
32,Michael,270,4,0.0
36,Michael,269,6,0.0
42,Michael,268,5,0.0
44,Michael,267,5,0.0
55,Michael,264,4,0.0
59,Michael,263,3,0.0


### win/loss ratio and win percent


In [117]:

dad_winloss = (len(dadwindf)/(len(daddf)-len(dadwindf)))
dad_winpercent = (len(dadwindf)/len(daddf)*100)

danielle_winloss = (len(daniellewindf)/(len(danielledf)-len(daniellewindf)))
danielle_winpercent = (len(daniellewindf)/len(danielledf)*100)

chris_winloss = (len(chriswindf)/(len(chrisdf)-len(chriswindf)))
chris_winpercent = (len(chriswindf)/len(chrisdf)*100)


ZeroDivisionError: division by zero

### need to account for division by zero --actually

i think percentage is enough. can circle back to ratio 


In [188]:
michael_winpercent = (len(michaelwindf)/len(michaeldf)*100)
dad_winpercent = (len(dadwindf)/len(daddf)*100)
danielle_winpercent = (len(daniellewindf)/len(danielledf)*100)
chris_winpercent = (len(chriswindf)/len(chrisdf)*100)

In [189]:
michael_winpercent

90.9090909090909

In [128]:
daniellewindf
danielledwindf

NameError: name 'daniellewindf' is not defined

In [145]:
dad_winpercent

100.0

In [146]:
danielle_winpercent

98.0

In [147]:
chris_winpercent

97.5609756097561

In [156]:
michaeldf

,player,game_num,score,LOSS
3,Michael,279,4,0.0
8,Michael,278,5,0.0
12,Michael,277,X,1.0
16,Michael,276,3,0.0
28,Michael,271,6,0.0
32,Michael,270,4,0.0
36,Michael,269,6,0.0
42,Michael,268,5,0.0
44,Michael,267,5,0.0
51,Michael,265,X,1.0


In [157]:
michaeldf

,player,game_num,score,LOSS
3,Michael,279,4,0.0
8,Michael,278,5,0.0
12,Michael,277,X,1.0
16,Michael,276,3,0.0
28,Michael,271,6,0.0
32,Michael,270,4,0.0
36,Michael,269,6,0.0
42,Michael,268,5,0.0
44,Michael,267,5,0.0
51,Michael,265,X,1.0


In [200]:
michaelwindf.score = michaelwindf.score.astype(int)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [201]:
michaelwindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 3 to 180
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   player    40 non-null     object 
 1   game_num  40 non-null     int64  
 2   score     40 non-null     int64  
 3   LOSS      40 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.6+ KB


In [205]:
michaelwindf

,player,game_num,score,LOSS
3,Michael,279,4,0.0
8,Michael,278,5,0.0
16,Michael,276,3,0.0
28,Michael,271,6,0.0
32,Michael,270,4,0.0
36,Michael,269,6,0.0
42,Michael,268,5,0.0
44,Michael,267,5,0.0
55,Michael,264,4,0.0
59,Michael,263,3,0.0


In [202]:
michaelwindf.score.sum()

180

In [204]:
michael_avg = michaelwindf.score.sum()/len(michaelwindf)
michael_avg

4.5

In [207]:
michaelwindf.score = michaelwindf.score.astype(int)
michael_avg = michaelwindf.score.sum()/len(michaelwindf)

daniellewindf.score = daniellewindf.score.astype(int)
danielle_avg = daniellewindf.score.sum()/len(daniellewindf)

chriswindf.score = chriswindf.score.astype(int)
chris_avg = chriswindf.score.sum()/len(chriswindf)

dadwindf.score = dadwindf.score.astype(int)
dad_avg = dadwindf.score.sum()/len(dadwindf)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [208]:
michael_avg

4.5

In [209]:
danielle_avg

4.020408163265306

In [210]:
chris_avg

4.225

In [211]:
dad_avg

3.8958333333333335

In [212]:
len(michaeldf)-len(michaelwindf)

4

In [213]:
michael_totalgamesplayed = len(michaeldf)
michael_winnum = len(michaelwindf)
michael_lossnum = len(michaeldf)-len(michaelwindf)

In [214]:
michael_totalgamesplayed

44

In [215]:
michael_winnum

40

In [216]:
michael_lossnum

4

In [218]:
michael_totalgamesplayed = len(michaeldf)
michael_winnum = len(michaelwindf)
michael_lossnum = len(michaeldf)-len(michaelwindf)

danielle_totalgamesplayed = len(danielledf)
danielle_winnum = len(daniellewindf)
danielle_lossnum = len(danielledf)-len(daniellewindf)

chris_totalgamesplayed = len(chrisdf)
chris_winnum = len(chriswindf)
chris_lossnum = len(chrisdf)-len(chriswindf)

dad_totalgamesplayed = len(daddf)
dad_winnum = len(dadwindf)
dad_lossnum = len(daddf)-len(dadwindf)